In [11]:
import pandas as pd
from tqdm import tqdm
from ast import literal_eval

In [2]:
df = pd.read_csv("data-dir/train_data.csv").drop("Unnamed: 0", axis=1)
df.head()

,Theme,Paragraph,Question,Answer_possible,Answer_text,Answer_start
0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,True,['2003'],[526]
1,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What album made her a worldwide known artist?,True,['Dangerously in Love'],[505]
2,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Who managed the Destiny's Child group?,True,['Mathew Knowles'],[360]
3,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyoncé rise to fame?,True,['late 1990s'],[276]
4,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What role did Beyoncé have in Destiny's Child?,True,['lead singer'],[290]


In [3]:
len(df)

75055

In [4]:
df = df.rename(columns={"Question": "question",
                   "Theme": "title",
                   "Paragraph": "context",
                   "Answer_possible": "answerable",
                   "Answer_start": "answer_start", 
                   "Answer_text": "answer_text", 
                   })

In [5]:
df["question"] = df["title"] + " " + df["question"]

In [6]:
df = df.drop(df.loc[(df.duplicated(subset=["question"], keep=False)) & (
        df["answerable"] == False)].index, axis=0).reset_index(drop=True)

In [7]:
len(df)

75038

In [8]:
# len(df.loc[(df.duplicated(subset=["Question"], keep=False)) & (df["Answer_possible"] == False)].sort_values("Question"))

In [9]:
# df = df.drop(df.loc[(df.duplicated(subset=["Question"], keep=False)) & (df["Answer_possible"] == False)].index, axis=0).reset_index(drop=True)

In [10]:
ques2idx = {ques: idx for idx, ques in enumerate(df["question"].unique())}
con2idx = {con: idx for idx, con in enumerate(df["context"].unique())}
title2idx = {title: idx for idx, title in enumerate(df["title"].unique())}

for i in tqdm(range(len(df))):
    df.loc[i, "question_id"] = ques2idx[df.iloc[i]["question"]]
    df.loc[i, "context_id"] = con2idx[df.iloc[i]["context"]]
    df.loc[i, "title_id"] = title2idx[df.iloc[i]["title"]]

100%|██████████| 75038/75038 [00:45<00:00, 1637.99it/s]


In [11]:
df["answer_start"] = df["answer_start"].apply(lambda x: literal_eval(x))

In [12]:
df["answer_start"].apply(lambda x: len(x)).describe()

count    75038.000000
mean         0.667995
std          0.470936
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: answer_start, dtype: float64

In [13]:
df["answer_start"] = df["answer_start"].apply(lambda x: x[0] if len(x) > 0 else "")

In [14]:
df["answer_text"] = df["answer_text"].apply(lambda x: literal_eval(x))

In [15]:
df["answer_text"].apply(lambda x: len(x)).describe()

count    75038.000000
mean         0.667995
std          0.470936
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: answer_text, dtype: float64

In [16]:
df["answer_text"] = df["answer_text"].apply(lambda x: x[0] if len(x) > 0 else "")

In [17]:
df["question_id"] = df["question_id"].astype(int)
df["context_id"] = df["context_id"].astype(int)
df["title_id"] = df["title_id"].astype(int)

In [18]:
df

,title,context,question,answerable,answer_text,answer_start,question_id,context_id,title_id
0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyoncé When did Beyonce leave Destiny's Child...,True,2003,526,0,0,0
1,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyoncé What album made her a worldwide known ...,True,Dangerously in Love,505,1,0,0
2,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyoncé Who managed the Destiny's Child group?,True,Mathew Knowles,360,2,0,0
3,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyoncé When did Beyoncé rise to fame?,True,late 1990s,276,3,0,0
4,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Beyoncé What role did Beyoncé have in Destiny'...,True,lead singer,290,4,0,0
...,...,...,...,...,...,...,...,...,...
75033,Matter,These quarks and leptons interact through four...,Matter How many quarks and leptons are there?,False,,,75023,15553,360
75034,Matter,These quarks and leptons interact through four...,Matter What model satisfactorily explains grav...,False,,,75024,15553,360
75035,Matter,These quarks and leptons interact through four...,Matter Mass and energy can always be compared ...,False,,,75025,15553,360
75036,Matter,"The term ""matter"" is used throughout physics i...",Matter Physics has broadly agreed on the defin...,False,,,75026,15554,360


In [19]:
df.dtypes

title           object
context         object
question        object
answerable        bool
answer_text     object
answer_start    object
question_id      int64
context_id       int64
title_id         int64
dtype: object

In [20]:
df.to_pickle("data-dir/data_prepared.pkl")

In [12]:
df = pd.read_pickle("data-dir/data_prepared.pkl")

In [13]:
df.to_csv("data-dir/data_prepared.csv", index=False)

In [ ]:
10807